In [44]:
import numpy as np
import os
import geopandas as gpd
import pandas as pd
from scipy import stats as scipy_stats
import scipy.signal
import rasterio
import rasterio.features
from rasterio import MemoryFile
from affine import Affine
from rasterio.plot import show
from rasterio.plot import show_hist
import matplotlib.pyplot as plt
import rasterstats
from scipy.ndimage import gaussian_filter
from shapely.geometry import Polygon, shape, MultiPolygon
import shapely
import subprocess as sp

# Import libs
import sys
module_path = os.path.abspath(os.path.join('..', 'libs'))
if module_path not in sys.path:
    sys.path.append(module_path)
from s3utils import *

In [3]:
S3R = S3Helper('roof-index')

# Read necessary_data
height = S3R.read_tif_from_s3_as_rio("missoula/geospatial/downtown_height.tif" ) 
height = S3R.read_tif_from_s3_as_rio("missoula/geospatial/downtown_dsm.tif" ) 
# bldgs = S3R.read_shp_from_s3_as_gpd("missoula/geospatial/downtown_flat_beta.zip").to_crs(6514)
flat = S3R.read_shp_from_s3_as_gpd("missoula/geospatial/contiguous_flat.zip").to_crs(6514)

In [74]:
out = []
for geom in flat['geometry']:
    interior_coords = []
    for interior in geom.interiors:
        interior_coords.append(interior.coords)
    
    out.append(interior_coords)

interiors = []
for geom in out:
    interiors.append(MultiPolygon([Polygon(geom[i]) for i, _ in enumerate(geom)]))    

areas = [x.area * 10.7639 for x in interiors]
flat['interiors'] = interiors
flat['int_area'] = areas